In [16]:
from openai import AzureOpenAI
from openai.types import Embedding
from dotenv import load_dotenv
import pytest
import os
import logging
import tiktoken 
from json import JSONDecodeError
from sklearn.metrics.pairwise import cosine_similarity
from cluestar import plot_text

load_dotenv('.env.shared')
load_dotenv('.env.secret')

test = os.getenv('OPENAI_API_KEY')
print(test)

6ccabd95d76e4f5084e6447e5f0be172


In [21]:
openai_client = AzureOpenAI(
    api_key=test,
    base_url=os.getenv('OPENAI_API_BASE'),
    api_version=os.getenv('OPENAI_API_VERSION')
)
print(openai_client.api_key)



6ccabd95d76e4f5084e6447e5f0be172


NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [46]:
system_prompt = """The user will provide you with a paragraph of text and a metadata string.
The data will follow this format:

METADATA: 
PARAGRAPH:

The paragraphs and metadata can be in different languages. They all come from the same domain of oil and gas exploration and production. 

You will create questions that can be answered from the text, alongside the answers to the questions.
The metadata should always be explicitly written in the questions and explicitly written in the answers.
You may also use exact sentences or shorten the sentences to phrases to generate examples.
You may paraphrase the content in sentences or change them slightly but you must preserve both language and style of individual texts.
If there are no examples to generate then create an empty list. 
Provide a confidence score between 0 and 1 for each of the question answer pairs where 1 is full confidence in the answer, and 0 is no confidence in the answer.

You are to produce a JSON formatted output, with JSON output only in the following form:

[{"Q": "<Question 1a>", "A": "<Answer 1b>", "C" : "<Confidence 1c>"}, 
 {"Q": "<Sentence 2a>", "A": "<Sentence 2b>", "C" : "<Confidence 2c>"},
 ...,
 {"Q": "<Sentence Na>", "A": "<Sentence Nb>", "C" : "<Confidence Nc>"}]

You are to determine yourself how many examples N you can come up with. Just make sure its always JSON formatted.
"""

In [43]:
user_prompt = lambda a, b: """
METADATA: """+a+""" 

PARAGRAPH: """+b 

metadata = "15/5-1"
paragraph = 'IV PETROPHYSICS AND WELL PRODUCTIVITY IV. 1. Loq Interpretation Investigation of the logs in well 15/5-1 showed hydro- carbon sand in the interval 3558-3614 RKBx, corre- sponding to total thickness of 56 m. No hydrocarbon water contact was identified.  The sand section may be subdivided into four separate pay zones, each zone being separated by thin impermeable layers. Pay zone Pay interval Thickness Avg. porosity Avg. SW m RKB 1 3558.6-3590.9 15 11 2 3594.8-3597.9 12 25 4 3608.8-3613.7 12 22 Total 14 14 IV. Observations The upper zone is the main zone, equivalent to 83% of the oil column. The cut-off criteria used in the log evaluation were: x The CP1 log is used as depth reference. RKB elevation ,.2-5m Refer Ficrui-e IV.1 and 7eservoir Fvaluation Display Section 8.'
prompt = user_prompt(metadata, paragraph)

In [32]:
user_prompt = 'IV PETROPHYSICS AND WELL PRODUCTIVITY IV. 1. Loq Interpretation Investigation of the logs in well 15/5-1 showed hydro- carbon sand in the interval 3558-3614 RKBx, corre- sponding to total thickness of 56 m. No hydrocarbon water contact was identified.  The sand section may be subdivided into four separate pay zones, each zone being separated by thin impermeable layers. Pay zone Pay interval Thickness Avg. porosity Avg. SW m RKB 1 3558.6-3590.9 15 11 2 3594.8-3597.9 12 25 4 3608.8-3613.7 12 22 Total 14 14 IV. Observations The upper zone is the main zone, equivalent to 83% of the oil column. The cut-off criteria used in the log evaluation were: x The CP1 log is used as depth reference. RKB elevation ,.2-5m Refer Ficrui-e IV.1 and 7eservoir Fvaluation Display Section 8.'

In [41]:
print(prompt)


METADATA: 15/5-1 

PARAGRAPH: IV PETROPHYSICS AND WELL PRODUCTIVITY IV. 1. Loq Interpretation Investigation of the logs in well 15/5-1 showed hydro- carbon sand in the interval 3558-3614 RKBx, corre- sponding to total thickness of 56 m. No hydrocarbon water contact was identified.  The sand section may be subdivided into four separate pay zones, each zone being separated by thin impermeable layers. Pay zone Pay interval Thickness Avg. porosity Avg. SW m RKB 1 3558.6-3590.9 15 11 2 3594.8-3597.9 12 25 4 3608.8-3613.7 12 22 Total 14 14 IV. Observations The upper zone is the main zone, equivalent to 83% of the oil column. The cut-off criteria used in the log evaluation were: x The CP1 log is used as depth reference. RKB elevation ,.2-5m Refer Ficrui-e IV.1 and 7eservoir Fvaluation Display Section 8.


In [47]:
# Load the shared environment variables, not secrets
load_dotenv("../.env.shared")
load_dotenv("../.env.secret")


openai_client = AzureOpenAI(
    api_version=os.environ["OPENAI_API_VERSION"],
    azure_endpoint=os.environ["OPENAI_API_BASE"],
    api_key=os.environ["OPENAI_API_KEY"]
)




completion = openai_client.chat.completions.create(model=os.environ["GPT432k_DEPLOYMENT"], 
                                            messages=[{"role": "system", 
                                                       "content": system_prompt}, 
                                                       {"role": "user", "content": prompt}])

In [48]:
print(completion.choices[0].message.content)

[{"Q": "What investigation was done in well 15/5-1?", "A": "Investigation of the logs in well 15/5-1 was done to show hydrocarbon sand in the interval 3558-3614 RKBx, corresponding to total thickness of 56 m. No hydrocarbon water contact was identified.", "C" : "1"}, 
 {"Q": "Can the sand section in well 15/5-1 be divided?", "A": "Yes, the sand section may be subdivided into four separate pay zones, each zone being separated by thin impermeable layers in well 15/5-1.", "C" : "1"} ,
 {"Q": "Which is the main zone in the well 15/5-1?", "A": "The upper zone is the main zone, equivalent to 83% of the oil column in well 15/5-1.", "C" : "1"},
 {"Q": "What were the cut-off criteria used in the log evaluation of well 15/5-1?", "A": "The cut-off criteria used in the log evaluation of well 15/5-1 were: the CP1 log is used as depth reference and RKB elevation", "C" : "1"}]


In [ ]:
import asyncio
import aiohttp

async def fetch(session, url, headers, data):
    async with session.post(url, headers=headers, data=data) as response:
        return await response.text()

async def fetch_all(urls, headers, data, loop):
    async with aiohttp.ClientSession(loop=loop) as session:
        results = await asyncio.gather(*[fetch(session, url, headers, data) for url in urls], return_exceptions=True)
        return results

prompts = {"prompt1": "Translate the following English text to French: '{text}'", "prompt2": "What is the weather like today?"}

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}',
}

data = {
    'model': 'text-davinci-002',
    'max_tokens': 60,
}

loop = asyncio.get_event_loop()
urls = [f"https://api.openai.com/v1/engines/davinci-codex/completions?prompt={prompt}" for prompt in prompts.values()]
responses = loop.run_until_complete(fetch_all(urls, headers, data, loop))

for response in responses:
    print(response)
